In [1]:
import util
import json
import matplotlib
#matplotlib.use('Agg')
import argparse
import copy
import glob
import h5py
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
from scipy.misc import imsave, imresize
from scipy.ndimage import imread
import os
from tqdm import tqdm

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DATA_PATH = 'data'                              # Where to store temporary files
PUBLIC_PATH = 'public'                          # Where to show graphs and diagnostics
EGOCENTRIC_IMAGE_FOLDER = 'egocentric_images'   # Where egocentric images are stored
TMP_PATH = 'tmp'  

In [3]:
feature_hash = 4023239360457093510
experiment_hash = 8841606304273805265

In [ ]:
print 'TRAINING MODEL'
print 'Loading data'

X = np.load('X_%d.npy' % feature_hash)
y = np.load('y_%d.npy' % feature_hash)
t = np.load('t_%d.npy' % feature_hash)
print("Load Done...")
assert X.shape[0] == y.shape[0] == t.shape[0]
split_mask = np.load('split_mask_%d.npy' % experiment_hash)


X_train = X[split_mask == 'train']
X_val = X[split_mask == 'val']
X_test = X[split_mask == 'test']

y_train = y[split_mask == 'train']
y_val = y[split_mask == 'val']
y_test = y[split_mask == 'test']

t_train = t[split_mask == 'train']
t_val = t[split_mask == 'val']
t_test = t[split_mask == 'test']

X_train = X_train.clip(0, 1)
X_val = X_val.clip(0, 1)
X_test = X_test.clip(0, 1)

assert X_train.shape[0] == y_train.shape[0] == t_train.shape[0]
assert X_val.shape[0] == y_val.shape[0] == t_val.shape[0]
assert X_test.shape[0] == y_test.shape[0] == y_test.shape[0]

((X_train, t_train), y_train) = util.balance((X_train, t_train), y_train)
((X_val, t_val), y_val) = util.balance((X_val, t_val), y_val)
((X_test, t_test), y_test) = util.balance((X_test, t_test), y_test)

assert X_train.shape[0] == y_train.shape[0] == t_train.shape[0]
assert X_val.shape[0] == y_val.shape[0] == t_val.shape[0]
assert X_test.shape[0] == y_test.shape[0] == y_test.shape[0]

TRAINING MODEL
Loading data


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.layers import Bidirectional
from keras.layers import LSTM



In [ ]:
print("Reshape X Train...")
tempLength = args.LOOKAHEAD + args.LOOKBEHIND + 1
X_train = X_train.reshape((X_train.shape[0], tempLength, X_train.shape[1]//tempLength))
X_val = X_val.reshape((X_val.shape[0], tempLength, X_val.shape[1]//tempLength))
X_test = X_test.reshape((X_test.shape[0], tempLength, X_test.shape[1]//tempLength))

model = Sequential()
model.add(Bidirectional(LSTM(3, return_sequences=False), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, W_regularizer=l2(args.C), activation='sigmoid'))
model.summary()
np.random.seed(args.RANDOM_SEED)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=args.LR), metrics=['accuracy'])
print 'Training model'
model.fit(X_train, y_train, nb_epoch=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[EarlyStopping(patience=3)])
model.save('model.h5')


In [ ]:
print 'Saving predictions'
X = np.load('X_%d.npy' % feature_hash)

if args.MODEL == 'keras':
    model.save('model_%d.h5' % experiment_hash)
    X = X.reshape((X.shape[0], tempLength, X.shape[1]//tempLength))
    y_prob = model.predict_proba(X)
    y_hat = model.predict_classes(X)
elif args.MODEL == 'svm':
    y_prob = 1.0 / (1.0 + np.exp(-model.decision_function(X)))
    y_hat = model.predict(X)
elif args.MODEL == 'logistic_regression':
    y_hat = model.predict(X)
    y_prob = model.predict_proba(X)[:, 1].flatten()
else:
    assert False, 'Unknown model: %d' % args.MODEL


np.save('y_prob_%d.npy' % experiment_hash, y_prob)
np.save('y_hat_%d.npy' % experiment_hash, y_hat)